<a href="https://colab.research.google.com/github/sasuraibito1125/google_colab/blob/main/Excel%E6%AF%94%E8%BC%83%E3%83%84%E3%83%BC%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 比較ツール

In [ ]:
!pip -qq install openpyxl

In [ ]:
# @title #### Excelファイル内容の正規化 { display-mode: "form" }

# @markdown 関数名：`normalize_excel_contents`

# @markdown キー項目：
# @markdown  * `sheet_name`
# @markdown  * `min_row`
# @markdown  * `max_row`
# @markdown  * `min_col`
# @markdown  * `max_col`
# @markdown  * `values`
def normalize_excel_contents(filename, verbose=False):
  '''Excelファイルの内容を正規化する。

    Parameters:
    -----------
      filename: 処理対象のExcelファイル名
      verbose: 冗長出力有効フラグ

    Returns:
    --------
      dict: キーは 'sheet_name', 'min_row', 'max_row', 'min_col', 'max_col', and 'values'。
            'sheet_name'の型は文字列、'values'は各セルの型の二次元リスト、それ以外は正の整数。
  '''
  from openpyxl import load_workbook
  wb = load_workbook(filename, read_only=True)
  sheetnames = wb.sheetnames
  result = []

  for name in sheetnames:
    normalized = {}
    normalized['sheet_name'] = name

    ws = wb[name]

    data_range = {
      'min_row': ws.min_row,
      'max_row': ws.max_row,
      'min_col': ws.min_column,
      'max_col': ws.max_column,
    }
    normalized['min_row'] = data_range['min_row']
    normalized['max_row'] = data_range['max_row']
    normalized['min_col'] = data_range['min_col']
    normalized['max_col'] = data_range['max_col']
    if verbose:
      print('行数:', data_range['max_row'] - data_range['min_row'],
            '列数:', data_range['max_col'] - data_range['min_col'])

    normalized['values'] = []
    for row in ws.iter_rows(**data_range):
      cells = []
      for cell in row:
        if verbose and cell.value:
          print(f'{cell.coordinate}({cell.row}, {cell.column})', cell.value)
        cells.append(cell.value)
      normalized['values'].append(cells)

    result.append(normalized)

  return result


In [ ]:
# @title #### 正規化されたExcelデータの差分を出力 { display-mode: "form" }

# @markdown 関数名：`diff_normalized_excel_data`

# @markdown キー項目：
# @markdown  * シート数
# @markdown  * シート名
# @markdown  * 最小行数
# @markdown  * 最大行数
# @markdown  * 最小列数
# @markdown  * 最大列数
# @markdown  * セル値
def diff_normalized_excel_data(first, second, verbose=False):
  '''正規化されたExcelデータの差分だけを出力する。

    Parameters:
    -----------
      first: 一つ目の正規化されたExcelデータ
      second: 二つ目の正規化されたExcelデータ
      verbose: 冗長出力有効フラグ

    Returns:
    --------
      TODO
  '''
  from itertools import zip_longest, count

  result = {}

  len_1st = len(first)
  len_2nd = len(second)
  if len_1st != len_2nd:
    result['シート数'] = (len_1st, len_2nd)
  if verbose:
    print('シート数が異なる' if 'シート数' in result else 'シート数が同じ')

  def diff_ws(i, s1, s2, key, result, result_key, verbose):
    if s1 and s2 and s1[key] != s2[key] or not s1 or not s2:
      diff = { i: (s1[key] if s1 else None, s2[key] if s2 else None) }

      if verbose:
        if not result_key in result:
          print(f'{result_key}が異なる')
        print(result_key, diff)

      if result_key in result:
        result[result_key].append(diff)
      else:
        result[result_key] = [diff]

  for i, s1, s2 in zip_longest(range(max(len_1st, len_2nd)), first, second):
    sheet_result = {}
    diff_ws(i, s1, s2, 'sheet_name', sheet_result, 'シート名', verbose)
    diff_ws(i, s1, s2, 'min_row', sheet_result, '最小行数', verbose)
    diff_ws(i, s1, s2, 'max_row', sheet_result, '最大行数', verbose)
    diff_ws(i, s1, s2, 'min_col', sheet_result, '最小列数', verbose)
    diff_ws(i, s1, s2, 'max_col', sheet_result, '最大列数', verbose)

    if verbose:
      for key in ['シート名', '最小行数', '最大行数', '最小列数', '最大列数']:
        if not key in sheet_result:
          print(f'{key}が同じ')

    # セル値チェックは両方ともあるものだけ
    if s1 and s2:
      for j, (r1, r2) in enumerate(zip(s1['values'], s2['values'])):
        for k, (c1, c2) in enumerate(zip(r1, r2)):
          if c1 != c2:
            diff = { (j, k): (c1, c2) }
            if 'セル値' in sheet_result:
              sheet_result['セル値'].append(diff)
            else:
              sheet_result['セル値'] = [diff]

    if 'シート' in result:
      result['シート'].append(sheet_result)
    else:
      result['シート'] = [sheet_result]

  return result


# ツール利用

#### 手順

1. 比較する2つのExcelファイルをアップロードする
2. 下記フォームにファイル名を設定する
3. ファイル全体を実行する

In [ ]:
# @title #### Excelファイル比較 { run: "auto", display-mode: "form" }
# @markdown Excelファイル名（1つ目）
filename1 = "foo.xlsx" # @param {type:"string"}
# @markdown Excelファイル名（2つ目）
filename2 = "bar.xlsx" # @param {type:"string"}

result = diff_normalized_excel_data(normalize_excel_contents(filename1),
                                    normalize_excel_contents(filename2))
result